<a href="https://colab.research.google.com/github/ilEnzio/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/ERLE_GRANGERII_DS18_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [ ]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [ ]:
import matplotlib.pyplot as plt
from collections import namedtuple

In [ ]:
# let's look at the df
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.describe()

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
count,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000
mean,1.201794,1.537149,40.750760,-73.972760,3579.585247,0.524838,0.478276,0.478276,0.447631,0.424852,0.415081,0.367085,0.052769,0.268452,0.185653,0.175902,0.132761,0.138394,0.102833,0.087203,0.060471,0.055206,0.051908,0.046193,0.043305,0.042711,0.039331,0.027224,0.026241
std,0.470711,1.106087,0.038954,0.028883,1762.430772,0.499388,0.499533,0.499533,0.497255,0.494326,0.492741,0.482015,0.223573,0.443158,0.388830,0.380741,0.339320,0.345317,0.303744,0.282136,0.238359,0.228385,0.221844,0.209905,0.203544,0.202206,0.194382,0.162738,0.159852
min,0.000000,0.000000,40.575700,-74.087300,1375.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,40.728300,-73.991800,2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,40.751700,-73.978000,3150.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,40.774000,-73.955000,4095.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.000000,8.000000,40.989400,-73.700100,15500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.describe(exclude="number")

,created,description,display_address,street_address,interest_level
count,48817,47392,48684,48807,48817
unique,48148,37853,8674,15135,3
top,2016-06-12 13:20:45,,Broadway,3333 Broadway,low
freq,3,1627,435,174,33946


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48817 entries, 0 to 49351
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bathrooms             48817 non-null  float64
 1   bedrooms              48817 non-null  int64  
 2   created               48817 non-null  object 
 3   description           47392 non-null  object 
 4   display_address       48684 non-null  object 
 5   latitude              48817 non-null  float64
 6   longitude             48817 non-null  float64
 7   price                 48817 non-null  int64  
 8   street_address        48807 non-null  object 
 9   interest_level        48817 non-null  object 
 10  elevator              48817 non-null  int64  
 11  cats_allowed          48817 non-null  int64  
 12  hardwood_floors       48817 non-null  int64  
 13  dogs_allowed          48817 non-null  int64  
 14  doorman               48817 non-null  int64  
 15  dishwasher         

In [ ]:
df.shape

(48817, 34)

In [ ]:
# I will call realeste rental df.  let's look at the types
realestate_rental_df = df.copy()


In [ ]:
realestate_rental_df.isnull().sum()

bathrooms                  0
bedrooms                   0
created                    0
description             1425
display_address          133
latitude                   0
longitude                  0
price                      0
street_address            10
interest_level             0
elevator                   0
cats_allowed               0
hardwood_floors            0
dogs_allowed               0
doorman                    0
dishwasher                 0
no_fee                     0
laundry_in_building        0
fitness_center             0
pre-war                    0
laundry_in_unit            0
roof_deck                  0
outdoor_space              0
dining_room                0
high_speed_internet        0
balcony                    0
swimming_pool              0
new_construction           0
terrace                    0
exclusive                  0
loft                       0
garden_patio               0
wheelchair_access          0
common_outdoor_space       0
dtype: int64

In [ ]:
realestate_rental_df.columns


Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft',
       'garden_patio', 'wheelchair_access', 'common_outdoor_space'],
      dtype='object')

In [ ]:
realestate_rental_df["dogs_allowed"].value_counts()

0    26965
1    21852
Name: dogs_allowed, dtype: int64

In [ ]:
# I'll create Total Rooms (Bed + Baths) and pets allow(Dogs or Cats)
# so lets just add together the rooms first
realestate_rental_df["total_rooms"] = realestate_rental_df["bedrooms"] + realestate_rental_df["bathrooms"] 

In [ ]:
realestate_rental_df.head(2)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,total_rooms
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0


In [ ]:
realestate_rental_df["cats_allowed"].head()

0    0
1    1
2    0
3    0
4    0
Name: cats_allowed, dtype: int64

In [ ]:
realestate_rental_df["dogs_allowed"].head()

0    0
1    1
2    0
3    0
4    0
Name: dogs_allowed, dtype: int64

In [ ]:
# now the pets_allow


pets_allowed = realestate_rental_df["cats_allowed"]


In [ ]:
pets_allowed.shape

(48817,)

In [ ]:
pets_allowed.head(20)

0     0
1     1
2     0
3     0
4     0
5     0
6     1
7     0
8     1
9     0
10    0
11    0
12    0
13    1
14    0
15    0
16    0
17    0
18    1
19    1
Name: cats_allowed, dtype: int64

In [ ]:
pets_allowed.value_counts()

0    25469
1    23348
Name: cats_allowed, dtype: int64

In [ ]:
# now go through and add values ... I just realized I could have just added the 
# two columns with np array broadcasting.  duh!!!!

for x in realestate_rental_df["cats_allowed"]:
  if realestate_rental_df.loc[x,"dogs_allowed"] == 1:
    pets_allowed.loc[x] = 1

pets_allowed.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0    0
1    1
2    0
3    0
4    0
Name: cats_allowed, dtype: int64

In [ ]:
pets_allowed.value_counts()

0    25469
1    23348
Name: cats_allowed, dtype: int64

In [ ]:
# it didn't change... so either the function didn't work or they are the same...

In [ ]:
pets_allowed = realestate_rental_df["cats_allowed"] + realestate_rental_df["dogs_allowed"]

In [ ]:
pets_allowed.head()

0    0
1    2
2    0
3    0
4    0
dtype: int64

In [ ]:
realestate_rental_df["pets_allowed"] = pets_allowed

In [ ]:
realestate_rental_df.head(2)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,total_rooms,pets_allowed
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2


## Create the Feature Matrix and Target Vector

In [ ]:
# create the Feature Matrix and Target
X_matrix  = realestate_rental_df.loc[:, ["pets_allowed", "total_rooms"]]
X_matrix.shape

(48817, 2)

In [ ]:
y_target = realestate_rental_df["price"].to_frame()
y_target.head()

,price
0,3000
1,5465
2,2850
3,3275
4,3350


## Split the data
Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test

In [ ]:
# check the datatype for created
realestate_rental_df["created"].head()


0    2016-06-24 07:54:24
1    2016-06-12 12:19:27
2    2016-04-17 03:26:41
3    2016-04-18 02:22:02
4    2016-04-28 01:32:41
Name: created, dtype: object

In [ ]:
# change it to datetime  
realestate_rental_df["created"] = realestate_rental_df["created"].astype("datetime64[ns]")
realestate_rental_df.created.head()

0   2016-06-24 07:54:24
1   2016-06-12 12:19:27
2   2016-04-17 03:26:41
3   2016-04-18 02:22:02
4   2016-04-28 01:32:41
Name: created, dtype: datetime64[ns]

In [ ]:
# now I should be able to access it via the year...tuple?? attribute??

In [ ]:
cond_apr_may_train = (realestate_rental_df["created"].dt.month == 4) | (realestate_rental_df["created"].dt.month == 5)
cond_apr_may_train


0        False
1        False
2         True
3         True
4         True
         ...  
49347    False
49348     True
49349     True
49350     True
49351     True
Name: created, Length: 48817, dtype: bool

In [ ]:
X_train, y_train = X_matrix.loc[cond_apr_may_train], y_target.loc[cond_apr_may_train]
X_val, y_val = X_matrix.loc[~cond_apr_may_train], y_target.loc[~cond_apr_may_train]

In [ ]:
X_train.shape

(31844, 2)

In [ ]:
X_val.shape

(16973, 2)

In [ ]:
y_train.shape

(31844, 1)

In [ ]:
y_val.shape

(16973, 1)

## Establish a Baseline
Basically just get a mean (MAE) - Naive model


In [ ]:
# lets get an average sales price
guess_price = realestate_rental_df["price"].mean()
guess_price

3579.5852469426636

In [ ]:
errors = guess_price - realestate_rental_df["price"]
errors.head()

0     579.585247
1   -1885.414753
2     729.585247
3     304.585247
4     229.585247
Name: price, dtype: float64

In [ ]:
# make function that will do it for us
def get_naive_baseline(target_feature):
  guess = target_feature.mean()
  errors = guess - target_feature
  mean_absolute_error = errors.abs().mean()

  naive_baseline = namedtuple("baseline_info", ["guess", "errors", "MAE"])

  return naive_baseline(guess, errors, mean_absolute_error)

In [ ]:
nbaseline = get_naive_baseline(realestate_rental_df["price"])


In [ ]:
print(nbaseline.guess)
print(nbaseline.MAE)
print(nbaseline.errors.shape)

3579.5852469426636
1201.532252154397
(48817,)


In [ ]:
# Get the MAE - no love for the STD... :( lol 
mean_absolute_error = errors.abs().mean()
mean_absolute_error

1201.532252154397

In [ ]:
# so...
print(f"If we priced the rentals for every condo at ${guess_price},")
print(f"we would be off by ${mean_absolute_error} on average.")

If we priced the rentals for every condo at $3579.5852469426636,
we would be off by $1201.532252154397 on average.


In [ ]:
X_train.shape

(31844, 2)

## Fit a linear regression model with at least two features.

I'll just used the two new features as my Feature Matrix - X_Matrix
and Rental price as they Target vector. 
> Indented block



In [ ]:
# lets bring in sci-kit learn and instantiate an object
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# our function would need:
#  X_matrix and y_target, and the dependent variable or correlate

def linear_regress_predict(X_train, y_train, X_val, y_val, correlate):

  # X_matrix and y_target
  #  
  # have already split it...
  
  # then it need to instantiate a linear regression object
  lr = LinearRegression()

# then it should fit the model to the trainig data
  lr.fit(X_train, y_train)

  # Then it can make predictions and store them as named tuples:
  # training_MAE, testing_MAE, response_variable
  # now make the predictions

  response_variable = lr.predict([correlate])
  response_var_column = lr.predict(X_train)

  training_MAE = mean_absolute_error(y_train, lr.predict(X_train))
  testing_MAE = mean_absolute_error(y_val, lr.predict(X_val))

  # it can also return the coeffient and intercept
  # coefficient, intercept 

  coefficient = lr.coef_
  intercept = lr.intercept_

  # I think we can also get RSME,  𝑅^2
  model_rmse = mean_squared_error(y_train, response_var_column, squared=False)
  model_r2 = r2_score(y_train, response_var_column)

  model_prediction = namedtuple("prediction_info", ["response_variable",
                                                    "training_MAE", "testing_MAE",
                                                    "model_RMSE", "model_r2_score",
                                                    "coefficient", 
                                                    "intercept"])
  return model_prediction(response_variable[0], training_MAE, testing_MAE,
                          model_rmse, model_r2, coefficient, intercept)

In [ ]:
# my correlate will be pets_allowed 0-2, total rooms 0-??

pets_rooms_value = [[1, 3]]


In [ ]:
prediction = linear_regress_predict(X_train, y_train, X_val, y_val, [2, 4] )

In [ ]:
print(f"Prediction: {prediction.response_variable}")

print(f"Model RMSE: {prediction.model_RMSE}")
print(f"Model R^2: {prediction.model_r2_score}")



print(f"Coefficient: {prediction.coefficient}")
print(f"Intercept: {prediction.intercept}")
print(f"Training MAE: {prediction.training_MAE}")
print(f"Testing MAE: {prediction.testing_MAE}")

Prediction: [4710.78711065]
Model RMSE: 1337.2460588780232
Model R^2: 0.424086708236337
Coefficient: [[100.97408923 809.54448582]]
Intercept: [1270.6609889]
Training MAE: 888.2758992592323
Testing MAE: 905.2782096048717
